In [5]:
import tensorflow as tf
import contractions
import re

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [4]:
tf.version.VERSION

'2.12.0'

In [6]:
class TextProcessor:

    def clean_text(self, text: str):
        text = str(text).lower()
        text = re.sub(r"&.[1-9]+;"," ", str(text))
        text=re.sub("(\\t)", ' ', str(text))
        text=re.sub("(\\r)", ' ', str(text))
        text=re.sub("(\\n)", ' ', str(text))
        text=re.sub("(__+)", ' ', str(text))
        text=re.sub("(--+)", ' ', str(text))
        text=re.sub("(~~+)", ' ', str(text))
        text=re.sub("(\+\++)", ' ', str(text))
        text=re.sub("(\.\.+)", ' ', str(text))
        
        # fix contractions to base form
        text = contractions.fix(text)

        #remove special tokens <>()|&©ø"',;?~*!
        text=re.sub(r"[<>()|&©ø\[\]\'\",;?~*!]", ' ', str(text)).lower()

        # CNN mail data cleanup
        text=re.sub("(mailto:)", ' ', str(text)) #remove mailto:
        text=re.sub(r"(\\x9\d)", ' ', str(text)) #remove \x9* in text
        text=re.sub("([iI][nN][cC]\d+)", 'INC_NUM', str(text)) #replace INC nums to INC_NUM
        text=re.sub("([cC][mM]\d+)|([cC][hH][gG]\d+)", 'CM_NUM', str(text)) #replace CM# and CHG# to CM_NUM

        # url replacement into base form
        try:
            url = re.search(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', str(text))
            repl_url = url.group(3)
            text = re.sub(r'((https*:\/*)([^\/\s]+))(.[^\s]+)',repl_url, str(text))
        except:
            pass


        # handle dot at the end of words
        text=re.sub("(\.\s+)", ' ', str(text))
        
        text=re.sub("(\-\s+)", ' ', str(text)) #remove - at end of words(not between)
        text=re.sub("(\:\s+)", ' ', str(text)) #remove : at end of words(not between)

        #remove multiple spaces
        text=re.sub("(\s+)",' ',str(text))

        # apply lowercase again
        text = text.lower().strip()
        
        # remove trailing dot, we will apply end of sequence anyway
        text = re.sub("(\.)$", '', str(text)).strip()

        return text

    def apply_special_tokens(self, text):
        text = str(text).strip()
        text = "<sos> " + str(text).strip() + " <eos>"
        return text

    def remove_special_tokens(self, text):
        text = text.lower()
        text = text.replace("<sos>", "").replace("<eos>", "")
        text = text.replace("<unk>", "##")
        text = text.strip()
        return text

In [7]:
class Summarizer:
    
    # Initialize using SummarizationModule or loaded graph tf module
    def __init__(self, module: tf.Module):
        self.module = module
        self.processor = TextProcessor()
        
    def summarize(self, text: str):
        prepared = self.processor.apply_special_tokens(self.processor.clean_text(text))
        output_text, output_tensor, weights = self.module.predict(tf.constant([prepared]))
        return prepared, self.processor.remove_special_tokens(bytes.decode(output_text.numpy())), output_tensor, weights
    
    # Shorthand for text output only
    def __call__(self, text: str):
        return self.summarize(text)[1]

In [3]:
!ls

inference-testing-mac.ipynb lyric-snow


In [4]:
model = tf.saved_model.load("lyric-snow")

Metal device set to: Apple M1


2023-04-28 18:45:04.750615: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [8]:
s = Summarizer(model)

In [11]:
s("Hello there.")

NotFoundError: Graph execution error:

2 root error(s) found.
  (0) NOT_FOUND:  No registered 'ExpandDims' OpKernel for 'GPU' devices compatible with node {{node text_vectorization_1/StringSplit_1/stack}}
	 (OpKernel was found, but attributes didn't match) Requested Attributes: T=DT_STRING, Tdim=DT_INT32, _XlaHasReferenceVars=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"
	.  Registered:  device='XLA_CPU_JIT'; Tdim in [DT_INT32, DT_INT64]; T in [DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, 930109355527764061, DT_HALF, DT_UINT32, DT_UINT64, DT_FLOAT8_E5M2, DT_FLOAT8_E4M3FN]
  device='DEFAULT'; T in [DT_HALF]; Tdim in [DT_INT32]
  device='DEFAULT'; T in [DT_HALF]; Tdim in [DT_INT64]
  device='DEFAULT'; T in [DT_BFLOAT16]; Tdim in [DT_INT32]
  device='DEFAULT'; T in [DT_BFLOAT16]; Tdim in [DT_INT64]
  device='DEFAULT'; T in [DT_FLOAT]; Tdim in [DT_INT32]
  device='DEFAULT'; T in [DT_FLOAT]; Tdim in [DT_INT64]
  device='DEFAULT'; T in [DT_DOUBLE]; Tdim in [DT_INT32]
  